# Chapter 3.4 Onwards

## 3.38
1.

$(peter,paul,mary)=\frac{100+10-20}{2}=45$

$(peter,mary,paul)=\frac{100+10}{2}-20=35$

$(paul,peter,mary)=\frac{100-20+10}{2}=45$

$(paul,mary,peter)=\frac{100-20}{2}+10=50$

$(mary,peter,paul)=\frac{100}{2}+10-20=40$

$(mary,paul,peter)=\frac{100}{2}-20+10=40$

2.

```
|    Peter    |     Paul    | Bank |    Mary    |
|:-----------:|:-----------:|:----:|:----------:|
|             |             |  100 |            |
| Access: 100 | Access: 100 |      | Access:100 |
|   New: 100  |   New: 80   |      |   New: 50  |
|   Set: 110  |             |      |            |
|             |             | 110  |            |
|             |             |      | Set: 50    |
|             |             | 50   |            |
|             | Set: 80     |      |            |
|             |             | 80   |            |


|    Peter    |     Paul    | Bank |    Mary    |
|:-----------:|:-----------:|:----:|:----------:|
|             |             |  100 |            |
| Access: 100 | Access: 100 |      | Access:100 |
|   New: 110  |   New: 80   |      |   New: 50  |
|             |             |      |   Set: 50  |
|             |             | 50   |            |
|             |   Set: 80   |      |            |
|             |             | 80   |            |
|  Set: 110   |             |      |            |
|             |             | 110  |            |
```

## 3.39

In [ ]:
; n.b make-serializer not implemented yet
(define x 10)
(define s (make-serializer))
(parallel-execute
 (lambda ()
   (set! x ((s (lambda () (* x x)))))) ; P1
 (s (lambda () (set! x (+ x 1))))) ; P2

Possible values:
* `101`: 
    * $P_1$ sets `x` to `100`
    * $P_2$ increments `x` to `101`
* `121`:
    * $P_2$ increments `x` to `11`
    * $P_1$ sets `x` to `x` times `x`
* `100`:
    * $P_1$ accesses `x` (twice)
    * $P_2$ sets `x` to `11`
    * $P_1$ sets `x`

$P_2$ can interleave between the *computation* and *assignment* in $P_1$.

## 3.40

In [ ]:
(define x 10)
(parallel-execute
 (lambda () (set! x (* x x)))
 (lambda () (set! x (* x x x))))

Possible values:
* `100` (Steps 1 and 2 can be swapped):
    1. $P_1$ accesses `x` (`2` times)
    2. $P_2$ accesses `x` (`3` times)
    3. $P_2$ sets `x` to `1000`
    4. $P_1$ sets `x` to `100`
* `1000` (Steps 1 and 2 can be swapped):
    1. $P_1$ accesses `x` (`2` times)
    2. $P_2$ accesses `x` (`3` times)
    3. $P_1$ sets `x` to `100`
    4. $P_2$ sets `x` to `1000`
* `10000`: 
    *  
        1. $P_1$ accesses `x` (`1` time)
        2. $P_2$ accesses `x` (`3` times)
        3. $P_2$ sets `x` to `1000`
        4. $P_1$ accesses `x` (`1` time) -> `1000`
        5. $P_1$ sets `x` to `10000`
    * 
        1. $P_2$ accesses `x` (`2` times)
        2. $P_1$ accesses `x` (`2` times)
        3. $P_1$ sets `x` to `100`
        4. $P_2$ accesses `x` (`1` time) -> `100`
        5. $P_2$ sets `x` to `10000` -> `(* 10 10 100)`
* `100000`:
    1. $P_2$ accesses `x` (`1` time)
    2. $P_1$ accesses `x` (`2` times)
    3. $P_1$ sets `x` to `100`
    4. $P_2$ accesses `x` (`2` times) -> `100`
    5. $P_2$ sets `x` to `100000` -> `(* 10 100 100)`
* `1000000`:
    * 
        1. $P_2$ accesses `x` (`3` times)
        2. $P_2$ sets `x` to `1000`
        3. $P_1$ accesses `x` (`2` times)
        4. $P_1$ sets `x` to `1000000` -> `(* 1000 1000)`
    * 
        1. $P_1$ accesses `x` (`2` times)
        2. $P_1$ sets `x` to `100`
        3. $P_2$ accesses `x` (`3` times) -> `100`
        4. $P_2$ sets `x` to `1000000` -> `(* 100 100 100)`
       

In [ ]:
(define x 10)
(define s (make-serializer))
(parallel-execute
 (s (lambda () (set! x (* x x))))
 (s (lambda () (set! x (* x x x)))))

`1000000` is the only possible value as $P_1$ and $P_2$ cannot interleave due to the serializer. Execution must follow one of the two paths shown for the `1000000` value in part 1.

## 3.41
No, unserialized access to the balance is acceptable because, in the given implementation, accessing the balance does not alter it's value and is completed in **a single step**.

If accessing the balance required multiple steps, then an unserialized implementation could result in anomalous behaviour as other processes could be interleaved between the steps. 

## 3.42

In [ ]:
; creates new serialized procedure on each withdraw/deposit message
(define (make-account balance)
  (define (withdraw amount)
    (if (>= balance amount)
        (begin (set! balance
                     (- balance amount))
               balance)
        "Insufficient funds"))
  (define (deposit amount)
    (set! balance (+ balance amount))
    balance)
  (let ((protected (make-serializer)))
    (define (dispatch m)
      (cond ((eq? m 'withdraw)
             (protected withdraw))
            ((eq? m 'deposit)
             (protected deposit))
            ((eq? m 'balance)
             balance)
            (else (error "make-account.dispatch"
                         "Unknown request: ~a"
                         m))))
    dispatch))

; returns same serialized procedure that was created at same time as account
(define (make-account balance)
  (define (withdraw amount)
    (if (>= balance amount)
        (begin (set! balance
                     (- balance amount))
               balance)
        "Insufficient funds"))
  (define (deposit amount)
    (set! balance (+ balance amount))
    balance)
  (let ((protected (make-serializer)))
    (let ((protected-withdraw
           (protected withdraw))
          (protected-deposit
           (protected deposit)))
      (define (dispatch m)
        (cond ((eq? m 'withdraw)
               protected-withdraw)
              ((eq? m 'deposit)
               protected-deposit)
              ((eq? m 'balance)
               balance)
              (else (error "make-account.dispatch"
                           "Unknown request: ~a"
                           m))))
        dispatch)))

Yes, the change is safe - there is no difference in the concurrency that is allowed because the procedures are still handled by the **same serializer**. 

## 3.43
When run sequentially, there can be no interleaving of operations between the processes. For two accounts `A` and `B` `(exchange A B)` maintains the overall sum of the balances between `A` and `B`, with the balance values remaining the same but swapped to the other account. 

When implemented using the first version of the account-exchange program, the operations between the processes can be interleaved - violating this property. For example:

```scheme
(define (exchange account1 account2)
  (let ((difference (- (account1 'balance)
                       (account2 'balance))))
    ((account1 'withdraw) difference)
    ((account2 'deposit) difference)))

(define a1 (make-account 10))
(define a2 (make-account 20))
(define a3 (make-account 30))

(parallel-execute
 (exchange a2 a1) ; P1
 (exchange a3 a1) ; P2
```

* $P_1$ computes `difference -> 10`
* $P_2$ computes `difference -> 10`
* $P_2$ withdraws `difference` from `a3`
    * `(a3 'balance) -> 20`
* $P_2$ deposits `difference` into `a2`
    * `(a2 'balance) -> 30`
* $P_1$ withdraws `difference` from `a2`
    * `(a2 'balance) -> 20`
* $P_1$ deposits `difference` into `a1` 
    * `(a1 'balance) -> 20`
* Final balances:
    * `a1 -> 20`
    * `a2 -> 20`
    * `a3 -> 20`

The overall sum of the balances has remained the same, as each `exchange` adds/subtracts an equal amount between accounts. However, the balance *values* have changed and the expected `exchange` behaviour has not been observed.

## 3.44

In [ ]:
(define
  (transer from-account to-account amount)
  ((from-account 'withdraw) amount)
  ((to-account 'deposit) amount))

Louis is wrong, with the constraint that `deposit` and `withdraw` procedures of the accounts are serialized, `transfer` makes no calls to unserialized procedures that interact with the account *state*. This means that the `deposit` and `withdraw` operations within `transfer` cannot interleave when run in parallel.

The `exchange` procedure made calls to the `balance` procedure on the account, which is *unserialized*. This allows for interleaving between parallel executions of `exchange`.

## 3.45
In Louis's implementation, the `withdraw` and `deposit` operations are serialized *twice* by then the *same* serializer - by `serialized-exchange` and by `dispatch` within the `account`s. `serialized-exchange` will never terminate as `withdraw` and `deposit` within `exchange` will wait until the execution of `serialized-exchange` is complete, which depends on those procedures executing.

## 3.46

In [2]:
(define (make-mutex)
  (let ((cell (list #f)))
    (define (the-mutex m)
      (cond ((eq? m 'acquire)
             (if (test-and-set! cell)
                 (the-mutex 'acquire))) ;retry
            ((eq? m 'release) (clear! cell))))
    the-mutex))
(define (clear! cell) (set-car! cell #f))

(define (test-and-set! cell)
  (if (car cell)
      #t
      (begin (set-car! cell #t)
             #f)))

|          P1          | cell |          P2          |
|:--------------------:|:----:|:--------------------:|
|         -            | `#f` |          -           |
|  `(car cell) -> #f`  |   -  |  `(car cell) -> #f`  |
| `(set-car! cell #t)` |   -  |          -           |
|         -           | `#t` | `(set-car! cell #t)` |
|         -            | `#t` |         -            |

## 3.47

In [4]:
; in terms of mutexes
(define (make-semaphore n)
  (let ((mutex (make-mutex))
        (current-processes 0))
    (define (the-semaphore m)
      (cond ((eq? m 'acquire)
             (mutex 'acquire)
             (if (< current-processes n)
                 (begin (set! current-processes (+ current-processes 1))
                        (mutex 'release))
                 (begin (mutex 'release)
                        (the-semaphore 'acquire)))) ; retry
            ((eq? m 'release)
             (mutex 'acquire)
             (set! current-processes (- current-processes 1))
             (mutex 'release))
            (else (error "the-semaphore"
                         "Unknown action: ~a"
                         m))))
    the-semaphore))

; in terms of atomic test-and-set! operations
(define (make-semaphore n)
  (let ((cell (list #f))
        (current-processes 0))
    (define (the-semaphore m)
      (cond ((eq? m 'acquire)
             (if (test-and-set! cell)
                 (the-semaphore 'acquire)) ; retry
             (cond ((> current-processes n)
                    (clear! cell)
                    (the-semaphore 'acquire))
                   (else
                    (set! current-processes (+ current-processes 1))
                    (clear ! cell))))
            ((eq? m 'release)
             (if (test-and-set! cell)
                 (the-semaphore 'release))
             (set! current-processes (- current-processes 1))
             (clear! cell))
            (else (error "the-semaphore"
                         "Unkown action: ~a"
                         m))))
    the-semaphore))